In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from data_gen import equity_data

In [5]:
#시장의 각종 anomaly를 리스크 프리미아(스마트 베타)
# 머신러닝
# 시스템 

In [6]:
perfdate11=(pd.datetime.today() - pd.tseries.offsets.BDay(6)).strftime('%Y-%m-%d')
perfdate1="2017-12-30"
perfdate2="2015-12-30"
perfdate3="2013-12-30"
perfdate4="1990-12-30"

In [7]:
name1 = 'tests'

# Read Data

In [8]:
ERet, Eindex = equity_data()

In [9]:
RETfinal=ERet
RET = ERet
Ret = ERet

# ECA
** Equity CArry 전략

총 인덱스 갯수는 14개 인데 이중 아시아 관련 4개를 따로 빼서 Cross Sectional을 하고 나머지 10개에 대해서 Cross Sectional을 따로 함. 이유는 걍 지시 사항이였다고.

In [10]:
columns1 = ERet.columns[:10]
columns2 = ERet.columns[10:]
columns1, columns2

(Index(['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX'], dtype='object'),
 Index(['NKY', 'AS51', 'HSI', 'SG'], dtype='object'))

In [11]:
# carry is monthly series
carry = pd.read_csv('./data/carry-dm.csv', index_col='Unnamed: 0', parse_dates= True)#, header = True)
carry['DAX'] = np.nan # why DAX NA????

In [12]:
minobs1=12
minobs=60

nopos=0.4##middle section->neutral zone


In [14]:
def get_tsrv_csrv(carry, CS = 0.35):
    #RV rank
    # carry 수치를 score로 rank 생성
    RV = carry.iloc[minobs1-1:]
    
    # Get RVrank for Time Series
    RVrank = RV.iloc[minobs1-1:]* 0
    for i in range(len(RV) - minobs1 +1):
        data=RV.iloc[:(minobs1+i)].rank(pct = True)
        RVrank.iloc[i] = data.iloc[-1][(len(data) - data.isna().sum() )>= minobs1]

    #CSRV
    RV1 = RV.iloc[minobs1-1:]

    truecount = np.round((RV1.shape[-1]-RV1.isna().sum(1)) * CS)
    truecount = np.repeat(truecount.values, len(RV1.columns)).reshape(RV1.shape)

    CSRV = RV1.rank(1, method = 'first')
    CSRV1 = RV1.rank(1, method = 'first', ascending = False)

    CSRVpos = CSRV* 0

    CSRVpos[CSRV<= truecount] = -1
    CSRVpos[CSRV1<= truecount] = 1

    CSRV = CSRVpos

    CSRV.fillna(0, inplace = True)

    # TSRV
    TSRV = RVrank *0

    TSRV[RVrank > (nopos+(1-nopos)/2)] = 1
    TSRV[RVrank < ((1-nopos)/2)] = -1
    return TSRV, CSRV


In [15]:
TSRV1, CSRV1 = get_tsrv_csrv(carry[columns1])
TSRV2, CSRV2 = get_tsrv_csrv(carry[columns2])

In [16]:
TSRV = pd.concat([TSRV1, TSRV2], 1)
CSRV = pd.concat([CSRV1, CSRV2], 1)

In [17]:
TSRV_mon = (TSRV.index+pd.tseries.offsets.BDay(1)).strftime('%Y-%m')
start = TSRV_mon[0]

CSRVtemp = TSRVtemp = ERet[start:]* np.nan

TSRVtemp = TSRV.reindex(TSRVtemp.index)
TSRVtemp.iloc[0] = TSRV.iloc[0]
TSRVtemp.ffill(inplace = True)

CSRVtemp = CSRV.reindex(CSRVtemp.index)
CSRVtemp.iloc[0] = CSRV.iloc[0]
CSRVtemp.ffill(inplace = True)

TSRV=TSRVtemp
CSRV=CSRVtemp
Ret=RETfinal


In [ ]:
# fx를 날짜를 기준으로 맞췄다.

In [18]:
def ex_ante_vol(df_ret, todate, weights, lookback=130, shrink_corr=0.9):
    '''
    Args:
        df_ret: pandas DataFrame of returns
        todate: date to calculate ex ante vol
        weights: weight of strategy before ex ante vol control
        lookback: number of days to consider for calculating cov
        shrink_corr: weight to shrink correlation
    '''
    # setting data range
    todate_loc = df_ret.index.get_loc(todate)
    if todate_loc > lookback:
        fromdate_loc = todate_loc - lookback
    else:
        fromdate_loc = 0
    
    # calculate corr apply shrinkage
    corr = df_ret.iloc[fromdate_loc:todate_loc].corr()
    corr = corr * shrink_corr + np.diag(np.ones(len(df_ret.columns))) * (1.- shrink_corr)
    # calculate cov
    std = df_ret.iloc[fromdate_loc:todate_loc].std()
    cov = pd.DataFrame(np.diag(std).dot(corr).dot(np.diag(std)), index=RET.columns, columns=RET.columns)
    # calculate ex ante vol
    return np.sqrt(weights.dot(cov).dot(weights)) * np.sqrt(260)

class GRP_bt:
    '''
    '''
    def __init__(self, CSLS, RET, RB, min_vol, freq):
        '''
        '''
        self.CSLS = CSLS  # Cross Sectional Exposure Type. 'vol' means volatility allocation. 'notional' means notional. 
        self.RET = RET  # 
        self.RB = RB
        
        self.Assetvol = 0.02 # vol control parameter for individual assets
        self.Strategyvol = 0.02 # vol control parameter for whole strategy
        self.factorvol = 0.02
        self.factorsd = 260 # lookback period for calcualting factorvol
        self.assetsd = 90 # lookback period for calculating Assetvol
        self.statsd = 90 # lookback period for calcualting Strategyvol

        self.volband = 0.05 # if controled vol is within volband, do not change vol
        self.min_vol = min_vol # vol floor 0.04 for ir strategy, 0.15 for all otheres
        self.freq = freq # strategy frequency setting. it only applys to Cross Sectional backtesting now.
        
        # Calculating volatility for assets
        std = (self.RET.rolling(window=self.assetsd).std() * np.sqrt(260)).iloc[self.assetsd:]
        # Adjust calculated asset volatility
        self.std_adj = self.adjust_by_volband(std, self.volband, min_vol=self.min_vol)

    def adjust_by_volband(self, df_vol, volband, min_vol=None):
        for i in range(1, len(df_vol.index)):
            if df_vol.iloc[i].name.weekday() == self.RB:
                for c in range(len(df_vol.columns)):
                    # case1. if new std is null
                    if pd.isnull(df_vol.iloc[i, c]):
                        df_vol.iloc[i, c] = df_vol.iloc[i-1, c]
                    # case2. if new std within volband
                    if abs(df_vol.iloc[i, c] - df_vol.iloc[i-1, c]) < volband * df_vol.iloc[i-1, c] :
                        df_vol.iloc[i, c] = df_vol.iloc[i-1, c]
            else:
                df_vol.iloc[i] = df_vol.iloc[i-1]
        if min_vol is not None:
            df_vol[df_vol < min_vol] = min_vol

        return df_vol
    
    def ts_vol_control_backtest(self, TSRV):
        # Time Series 
        VCweight = self.Assetvol / self.std_adj
        VCTSpos = (VCweight * TSRV).loc[TSRV.index[0]:] # Vol Control Time Series Position
        VCTSpos.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        # Strategy level vol control
        # return of VC strategy
        Strategy = ((self.RET * VCTSpos.shift(1)).sum(axis=1)).loc[TSRV.index[0]:]
        # using average of expanding & moving window strategy risk
        Strategyrisk_expanding = (Strategy.expanding(min_periods=self.statsd).std()*np.sqrt(260)).iloc[self.statsd:]
        Strategyrisk_rolling = (Strategy.rolling(window=self.statsd).std()*np.sqrt(260)).iloc[self.statsd:]
        Strategyrisk = (Strategyrisk_expanding + Strategyrisk_rolling) / 2.
        # volband adjustment
        bufrisk = self.adjust_by_volband(Strategyrisk.copy().to_frame(), self.volband)
        # apply strategy level vol control to position
        statlev = self.Strategyvol / bufrisk.iloc[:,0]
        self.TSposition = VCTSpos.multiply(statlev, axis='index').iloc[self.statsd:]
        self.TSposition.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.TSposition.fillna(0, inplace=True)
        
        # calculate daily strategy return
        self.TSStrategy = (self.RET * self.TSposition.shift(1)).sum(axis=1).loc[TSRV.index[0]:]
        self.TSStrategylag = (self.RET * self.TSposition.shift(2)).sum(axis=1).loc[TSRV.index[0]:]
    
    def cs_vol_control_backtest(self, CSRV):
        if self.CSLS == 'vol':
            CSRV = (CSRV * self.Assetvol / self.std_adj).loc[CSRV.index[0]:]
        #elif self.CSLS != 'notional': # raise error
            
        # Calculating Ex-ante Vol and adjust to target vol which specified by 'Strategyvol'
        if self.freq == 'week':
            for i in range(len(CSRV.index)):
                if CSRV.iloc[i].name.weekday() == self.RB:
                    CSRV.iloc[i] = CSRV.iloc[i] * self.Strategyvol / ex_ante_vol(self.RET, CSRV.iloc[i].name, CSRV.iloc[i])
                else:
                    CSRV.iloc[i] = CSRV.iloc[i-1]
        elif self.freq == 'month':
            for i in range(len(CSRV.index)):
                if CSRV.iloc[i].name.month != CSRV.iloc[i-1].name.month:
                    CSRV.iloc[i] = CSRV.iloc[i] * self.Strategyvol / ex_ante_vol(self.RET, CSRV.iloc[i].name, CSRV.iloc[i])
                else:
                    CSRV.iloc[i] = CSRV.iloc[i-1]
        #else: # raise error
            
        # Strategy level vol control
        # return of VC strategy
        CSraw = (self.RET * CSRV.shift(1)).dropna(how='all').sum(axis=1)
        CSrisk = (CSraw.rolling(window=self.statsd).std()*np.sqrt(260)).iloc[self.statsd:]
        
        # volband adjustment
        cs_bufrisk = self.adjust_by_volband(CSrisk.to_frame(), self.volband)
        # apply strategy level vol control to position
        cs_statlev = self.Strategyvol / cs_bufrisk.iloc[:,0]
        self.CSposition = CSRV.multiply(cs_statlev, axis='index').iloc[self.statsd:]
        self.CSposition.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.CSposition.fillna(0, inplace=True)
        
        # calculate daily strategy return
        self.CSStrategy = (self.RET * self.CSposition.shift(1)).sum(axis=1).loc[CSRV.index[0]:]
        self.CSStrategylag = (self.RET * self.CSposition.shift(2)).sum(axis=1).loc[CSRV.index[0]:]

In [19]:
RB = 1
min_vol = .15
freq = 'month'

In [20]:
GRP_bt_test = GRP_bt('vol', ERet, RB, min_vol, freq)
GRP_bt_test.ts_vol_control_backtest(TSRV)

(1. + GRP_bt_test.TSStrategy).cumprod().plot()
(1. + GRP_bt_test.TSStrategylag).cumprod().plot()

ValueError: cannot reindex from a duplicate axis

In [ ]:
GRP_bt_test.cs_vol_control_backtest(CSRV)

(1. + GRP_bt_test.CSStrategy).cumprod().plot()
(1. + GRP_bt_test.CSStrategylag).cumprod().plot()